In [1]:
from requests import request
import json
import pandas as pd
import math
import numpy
import pickle
from pathlib import Path
import urllib3
import getpass
import warnings
import os 

## Accessing Models via API
Now, we can head over to SAS Viya to manage our model using the visual interface. But, you don't need to use the visual interface in SAS Viya if you don't want to! You can actually access most of the model management capabilities through the SAS open source package SASCTL. For whatever isn't included in the open source package, you can do through API!

In this section, we will use the SAS Viya APIs to look at one of our deployed models. To use this notebook on your own, make sure you have deployed one of your models, either through the visual interface or programmatically. To get started, we need to generate an authorization token.

In [2]:
with open('access_token.txt', 'r') as token:
	access_token = token.read()

In [3]:
deployed_name = "gb_model_viyaml"
hostname = "https://innovationlab.demo.sas.com"
headers = {'Authorization': 'Bearer ' + access_token}

url = hostname + "/microanalyticScore/modules/" + deployed_name + "/steps"

r = request('GET', url, headers = headers, verify="innovation_lab.pem")

url, r

('https://innovationlab.demo.sas.com/microanalyticScore/modules/gb_model_viyaml/steps',
 <Response [200]>)

In [4]:
data_path = "../../Data/cleaned_data/train_valid_test.csv"
numerical_features = ["CreditLineAge","DebtIncRatio","FICOScore","Inquiries6Mnths",
                      "LogAnnualInc","RevBalance","RevUtilization","Installment","InterestRate"]
categorical_features = ["CreditPolicy","Delinquencies2Yrs","PublicRecord","Purpose"]
features = numerical_features+categorical_features
target = "Default"


test_data = pd.read_csv(data_path)
test_data = test_data[test_data["_PartInd_"]==3].reset_index(drop=True)

with open("artifacts/data_prep.pkl", mode="rb") as f:
    data_prep = pickle.load(f)
    
x_test_prep = pd.DataFrame(data_prep.transform(test_data[features]), columns=features)

In [5]:
# Prepare payload
input_dict = x_test_prep.iloc[2,:].to_dict()
l = []
for k,v in input_dict.items():
    d = {}
    d["name"]=k.lower()
    d["value"]=v
    l.append(d)
payload_dict = {"inputs":l}
payload_str = f'{payload_dict}'.replace('\'','\"')
payload_str

'{"inputs": [{"name": "creditlineage", "value": 0.2711659181247951}, {"name": "debtincratio", "value": 2.1167826630698605}, {"name": "ficoscore", "value": 0.6282422545529979}, {"name": "inquiries6mnths", "value": -0.864307013574943}, {"name": "logannualinc", "value": -1.3402145125518632}, {"name": "revbalance", "value": 0.11215111173361521}, {"name": "revutilization", "value": 0.033144183084290506}, {"name": "installment", "value": -1.077988366909529}, {"name": "interestrate", "value": -1.4192111007956187}, {"name": "creditpolicy", "value": 1.0}, {"name": "delinquencies2yrs", "value": 0.0}, {"name": "publicrecord", "value": 0.0}, {"name": "purpose", "value": 0.0}]}'

In [6]:
# Send request
headers = {'Content-Type': 'application/vnd.sas.microanalytic.module.step.input+json', 
            'Authorization': 'Bearer ' + access_token}
    
url = hostname + '/microanalyticScore/modules/' + deployed_name + '/steps/score'

r = request('POST', url, data=payload_str, headers = headers, verify='innovation_lab.pem')
r.json()

{'links': [],
 'version': 2,
 'moduleId': 'gb_model_viyaml',
 'stepId': 'score',
 'executionState': 'completed',
 'metadata': {'module_id': 'gb_model_viyaml', 'step_id': 'score'},
 'outputs': [{'name': 'P_Default1', 'value': 0.47552758148545937},
  {'name': 'P_Default0', 'value': 0.5244724185145406},
  {'name': 'I_Default', 'value': '           0'},
  {'name': '_WARN_', 'value': '    '}]}